In [3]:
import numpy as np
import tensorflow
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import math

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [6]:
MODEL_PATH = '/saved_models/nlp_models'

In [7]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 30.2 MB/s 
     |████████████████████████████████| 1.4 MB 50.2 MB/s 
     |████████████████████████████████| 233 kB 63.9 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394486 sha256=664035df8c03ccf75accfe80b6a61a7660275c2e783c2b0d755ad2cdf0dc709f
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=153425 sha256=6a64770c4e4b52d9ff91153cc69b58abdb08213bc9e0e6737c55e902b18874a0
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7


###**Text preprocessing functions**

In [8]:
from __future__ import unicode_literals
from hazm import *
import re
import random
from string import punctuation

def text_preprocess(text):
  normalizer = Normalizer()
  text = normalizer.normalize(text)
  text = re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)
  return text

def text_scramble(text):
  words = text.split()
  random.shuffle(words)
  return ' '.join(words)


###**Loading and processing Corpus 1**
(VOA FARSI 2003-08)

In [9]:
path = 'data/voa_fa_2003-2008_orig.txt'
with open(path, 'r', encoding='utf-8') as f:
  text = ' '.join([line.strip() for line in f.readlines() if not line.startswith('#')])
  text = text.split('.')
  sents = random.sample(text, 25000)

In [10]:
MAX_LEN = 170
sents_cleaned = [text_preprocess(s) for s in sents]
sents_ready = filter(lambda s: len(s) < MAX_LEN, sents_cleaned)
sents_data = list(sents_ready)
random.shuffle(sents_data)
sents_data = sents_data[:20000]  # make dataset size 20000 for ease of computation

In [11]:
tokenizer = keras.preprocessing.text.Tokenizer(oov_token='<UNK>')
tokenizer.fit_on_texts(sents_data)
tokenizer.word_index['<PAD>'] = 0
last_idx = len(tokenizer.word_index) + 1
tokenizer.word_index['<SOS>'] = last_idx

In [12]:
temp = [len(each.split()) for each in sents_data]
print('maximum sentence length:', sorted(temp)[-1])

maximum sentence length: 38


### **building the dataset**

In [13]:
def create_shuffles(sent_list, m):
  """
  this function creates m random shuffles of the sentence
  """
  all_combs = []
  for sent in sent_list:
    comb_set = set([])
    for i in range(0, m + 1):
      comb_set.add(text_scramble(sent))
    all_combs.append((sent, comb_set))
  return all_combs

In [14]:
shuffle_pairs = create_shuffles(sents_data, 10)

In [15]:
shuffle_pairs[1][0], shuffle_pairs[1][1]

(' یک روزنامه چاپ آمریکا گزارش می\u200cدهد که مهندسان روسی با نقض ممنوعیتی که سازمان ملل متحد به اجرا گذاشته بود به عراق در ساختن موشک\u200cهای بالیستیک دور برد کمک کردند',
 {'آمریکا بالیستیک ساختن گذاشته که به متحد برد به مهندسان ملل عراق کمک گزارش می\u200cدهد موشک\u200cهای یک اجرا نقض در ممنوعیتی بود کردند که دور با روسی سازمان چاپ روزنامه',
  'آمریکا نقض می\u200cدهد ساختن به گزارش کمک بود در برد روسی بالیستیک دور که عراق با به یک موشک\u200cهای ملل سازمان که ممنوعیتی کردند روزنامه مهندسان چاپ متحد گذاشته اجرا',
  'دور می\u200cدهد گزارش که بود بالیستیک گذاشته یک عراق با چاپ سازمان ممنوعیتی موشک\u200cهای ملل اجرا کمک در آمریکا به که به روزنامه مهندسان متحد نقض ساختن کردند روسی برد',
  'روسی آمریکا که ساختن به در برد بالیستیک یک روزنامه کمک ملل به موشک\u200cهای که مهندسان چاپ با دور عراق می\u200cدهد گذاشته متحد سازمان گزارش ممنوعیتی بود اجرا نقض کردند',
  'روسی روزنامه ملل می\u200cدهد برد نقض چاپ گذاشته که اجرا به موشک\u200cهای آمریکا گزارش کمک که ممنوعیتی ساختن کردند با عراق مهندسان یک

In [16]:
def create_dataframe(pairs):
  """
  creates a dataframe with two columns of original sentence and it's shuffles
  """
  original = []
  shuffled = []
  for pair in pairs:
    for p in pair[1]:
      original.append(pair[0])
      shuffled.append(p)

  df_dict = { 'Shuffled': shuffled, 'Original': original}
  df = pd.DataFrame(df_dict)
  return df


In [17]:
df = create_dataframe(shuffle_pairs)
df

,Shuffled,Original
0,توانست کند خوب از معدود را دفع بازیکنان رضا بو...,ولی رضا ناصری که از معدود بازیکنان خوب ایران ...
1,ضربه ولی که بازیکنان دفع کند خوب ناصری را رضا ...,ولی رضا ناصری که از معدود بازیکنان خوب ایران ...
2,ایران بازیکنان توانست ناصری کند بود از ولی که ...,ولی رضا ناصری که از معدود بازیکنان خوب ایران ...
3,این ولی ناصری بازیکنان کند ضربه که را معدود ای...,ولی رضا ناصری که از معدود بازیکنان خوب ایران ...
4,خوب این که رضا ولی ضربه را کند ناصری توانست از...,ولی رضا ناصری که از معدود بازیکنان خوب ایران ...
...,...,...
216776,رایس از و سه آمریکا نخست وزیر رفته با اسرائیل ...,اهود اولمرت نخست وزیر اسرائیل برای دیداری سه ...
216777,اهود آمریکا واشنگتن ملاقات اولمرت و برای سه ها...,اهود اولمرت نخست وزیر اسرائیل برای دیداری سه ...
216778,کاندولیزا و امور روزه وزیر سه جمله رئیس اهود و...,اهود اولمرت نخست وزیر اسرائیل برای دیداری سه ...
216779,هائی جمهوری اهود و با نخست خارجه وزیر رئیس رفت...,اهود اولمرت نخست وزیر اسرائیل برای دیداری سه ...


In [20]:
df = df.sample(frac=1).reset_index(drop=True)
df

,Shuffled,Original
0,صلح به هفت به امنیت گفته‌اند روسیه شناسائی در ...,وزیران گروه هفت گفته‌اند تصمیم روسیه به شناسا...
1,ملاقات آنها کنند ضمن و سیاستمداران شهر ناتو از...,قرار است هیات اعزامی ناتو ضمن ملاقات با دولتم...
2,دولت است به توضیح ایران متوقف نشود دست خواستار...,آنها درباره اینکه دولت ایران گفته است اگر حکم...
3,ژانویه آمریکائی کشته جداگانه شدن نود در سه رسی...,دیروز با کشته شدن دو سرباز آمریکائی در دو حمل...
4,سپتامبر در در تولید ماه دهند تشکیل قراراست ۱۱ ...,قراراست ۱۱ عضو اوپک برای بررسی شرایط بازار نف...
...,...,...
216776,را کردند کسب رشته نقره با ۲۵۸۰۷ مدال ۲۵۷۳۲ این...,تیم باهاماس با ۲۵۷۳۲ مدال نقره و جامائیکا با ...
216777,تحلیگران میگویند وی بسیاری اما کم کاندیدائی شا...,اما بسیاری از تحلیگران سیاسی میگویند شانس وی ...
216778,حاشیه می‌گوید آفریقا خود کمک‌های به و دارند رس...,سازمان ملل متحد می‌گوید حدود ۳۰ میلیون نفردرک...
216779,ملاقات عضو در رابرتسون ایالت کلورادو موضوع ایو...,جرج رابرتسون این موضوع را روز پنجشنبه در پی م...


In [19]:
def create_dataset(df, train_percent=80):
  data_size = df.shape[0]
  count = math.floor(data_size * (train_percent/100))

  dataset = df['Shuffled'].values
  labels  = df['Original'].values

  train_data   = dataset[:count]
  train_labels = labels[:count]
  test_data   = dataset[count:]
  test_labels = labels[count:]

  return ((train_data, train_labels), (test_data, test_labels))




In [21]:
train_set, test_set = create_dataset(df)
X_train_text, Y_train_text = train_set
X_test_text, Y_test_text = test_set

###**making dataset ready for training**

In [22]:
sos_index = last_idx
def shift_output_sequence(seq_list):
  for seq in seq_list:
    seq.insert(0, sos_index)
  return seq_list


In [23]:
X_train_seq = tokenizer.texts_to_sequences(X_train_text)
Y_train_seq = tokenizer.texts_to_sequences(Y_train_text)
X_test_seq  = tokenizer.texts_to_sequences(X_test_text)
Y_test_seq  = tokenizer.texts_to_sequences(Y_test_text)

In [24]:
MAX_SEQ_LEN = 45  
X_train = keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=MAX_SEQ_LEN, padding='post')
Y_train = keras.preprocessing.sequence.pad_sequences(Y_train_seq, maxlen=MAX_SEQ_LEN, padding='post')

X_test  = keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=MAX_SEQ_LEN, padding='post')
Y_test  = keras.preprocessing.sequence.pad_sequences(Y_test_seq, maxlen=MAX_SEQ_LEN, padding='post')

X_train_decoder = keras.preprocessing.sequence.pad_sequences(shift_output_sequence(Y_train_seq), maxlen=MAX_SEQ_LEN, padding='post')
X_test_decoder  = keras.preprocessing.sequence.pad_sequences(shift_output_sequence(Y_test_seq),  maxlen=MAX_SEQ_LEN, padding='post')

In [25]:
X_train_decoder[:2], Y_train[:2]

(array([[24346,   702,    78,   427,  1012,   290,    90,     3,  2127,
           547,  3932,   627,  1339,   725,     6,     3,   138,     4,
           107,     2, 19667,     3,  3648,  6477,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0],
        [24346,    48,     8,   855,  2993,   354,   602,   250,    10,
          9341,   779,  5818,     4,  2251,   399,    71,     5,    81,
          9031,   643,    38,   969,    96,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0]],
       dtype=int32),
 array([[  702,    78,   427,  1012,   290,    90,     3,  2127,   547,
          3932,   627,  1339,   725,     6,     3,   138,     4,   107,
             2, 19667,     3,  3648,  6477,     0,     0,     0,     0,
             0,     0,     0,     0,    

In [26]:
X_train[:1],Y_train[:1]

(array([[  138,     3,   427,     3,   107,  1012,    90,  2127,     2,
          3932,     4,   290,     6,   702,     3,   627,  3648,   547,
         19667,  1339,   725,    78,  6477,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0]],
       dtype=int32),
 array([[  702,    78,   427,  1012,   290,    90,     3,  2127,   547,
          3932,   627,  1339,   725,     6,     3,   138,     4,   107,
             2, 19667,     3,  3648,  6477,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0]],
       dtype=int32))

In [27]:
VOCAB_SIZE = len(tokenizer.word_index)
VOCAB_SIZE

24346

###**Encoder-Decoder network**

In [31]:
encoder_embedding_size = 32
decoder_embedding_size = 32
lstm_units = 128

np.random.seed(42)
tf.random.set_seed(42)

encoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)

encoder_embedding = keras.layers.Embedding(input_dim=VOCAB_SIZE + 1,output_dim=encoder_embedding_size, input_length=MAX_SEQ_LEN, mask_zero=True)(encoder_input)

_, encoder_state_h, encoder_state_c = keras.layers.LSTM(lstm_units, dropout=0.1, return_state=True)(encoder_embedding)

encoder_state = [encoder_state_h, encoder_state_c]

decoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)

decoder_embedding = keras.layers.Embedding(input_dim=VOCAB_SIZE + 2, output_dim=decoder_embedding_size, mask_zero=True)(decoder_input)

decoder_lstm_output = keras.layers.LSTM(lstm_units, dropout=0.1, return_sequences=True)(decoder_embedding, initial_state=encoder_state)

decoder_output = keras.layers.Dense(VOCAB_SIZE + 1, activation="softmax")(decoder_lstm_output)

model = keras.models.Model(inputs=[encoder_input, decoder_input], outputs=[decoder_output])

optimizer = keras.optimizers.Nadam(learning_rate=0.008)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_6 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_4 (Embedding)        (None, None, 32)     779104      ['input_5[0][0]']                
                                                                                                  
 embedding_5 (Embedding)        (None, None, 32)     779136      ['input_6[0][0]']                
                                                                                            

In [32]:
history = model.fit([X_train, X_train_decoder], Y_train, epochs=50, validation_split=0.15, batch_size=512)

Epoch 1/50
288/288 [==============================] - 147s 487ms/step - loss: 2.8987 - accuracy: 0.1483 - val_loss: 2.4522 - val_accuracy: 0.2167
Epoch 2/50
288/288 [==============================] - 137s 476ms/step - loss: 2.2113 - accuracy: 0.2504 - val_loss: 1.9770 - val_accuracy: 0.2950
Epoch 3/50
288/288 [==============================] - 137s 475ms/step - loss: 1.8278 - accuracy: 0.3241 - val_loss: 1.6612 - val_accuracy: 0.3701
Epoch 4/50
288/288 [==============================] - 137s 476ms/step - loss: 1.5710 - accuracy: 0.3877 - val_loss: 1.4432 - val_accuracy: 0.4295
Epoch 5/50
288/288 [==============================] - 136s 473ms/step - loss: 1.3953 - accuracy: 0.4351 - val_loss: 1.2885 - val_accuracy: 0.4737
Epoch 6/50
288/288 [==============================] - 137s 476ms/step - loss: 1.2681 - accuracy: 0.4721 - val_loss: 1.1635 - val_accuracy: 0.5160
Epoch 7/50
288/288 [==============================] - 137s 475ms/step - loss: 1.1723 - accuracy: 0.5017 - val_loss: 1.0724 -

In [44]:
history = model.fit([X_train, X_train_decoder], Y_train, epochs=20, validation_split=0.15, batch_size=512)

Epoch 1/20
288/288 [==============================] - 146s 492ms/step - loss: 0.5306 - accuracy: 0.7214 - val_loss: 0.4031 - val_accuracy: 0.8048
Epoch 2/20
288/288 [==============================] - 137s 477ms/step - loss: 0.5271 - accuracy: 0.7229 - val_loss: 0.3998 - val_accuracy: 0.8064
Epoch 3/20
288/288 [==============================] - 137s 476ms/step - loss: 0.5240 - accuracy: 0.7240 - val_loss: 0.3966 - val_accuracy: 0.8080
Epoch 4/20
288/288 [==============================] - 137s 475ms/step - loss: 0.5214 - accuracy: 0.7250 - val_loss: 0.3927 - val_accuracy: 0.8094
Epoch 5/20
288/288 [==============================] - 136s 473ms/step - loss: 0.5180 - accuracy: 0.7263 - val_loss: 0.3903 - val_accuracy: 0.8108
Epoch 6/20
288/288 [==============================] - 137s 476ms/step - loss: 0.5146 - accuracy: 0.7277 - val_loss: 0.3868 - val_accuracy: 0.8109
Epoch 7/20
288/288 [==============================] - 138s 478ms/step - loss: 0.5126 - accuracy: 0.7286 - val_loss: 0.3856 -

**almost 100% accuracy on training data after 30 epochs**

In [40]:
model.save("sentence_model_v3.h5")

###**Simple seq2seq network**

**model evaluation on test data**

In [41]:
model = keras.models.load_model("sentence_model_v3.h5")

In [45]:
ids = np.argmax(model.predict([X_test[:1], X_test_decoder[:1]]), axis=-1)
ids, Y_test[:1]

(array([[   5,  306,  157, 2285,  341,   88,  125,  590,   25, 1703,   98,
            7,  147, 1484,   69,  356,  123,    5, 1419,   34,    6,   12,
           12,    6, 1321,  111,   54,  133,    0,  159,  159,  159,  159,
          159,  159,  159,  159,  159,  159,  159,  159,  159,  159,  159,
          159]]),
 array([[   5,  306,  157, 2285,  341,   88,  125,    9,   25, 1703,   98,
            2,   67, 1484,   69,  356,  123,    5, 2207,   34,    7, 1323,
          417,  153, 1321,  111,   54,  133,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0]], dtype=int32))

In [46]:
print("Evaluate on test data")
results = model.evaluate([X_test, X_test_decoder], Y_test, batch_size=512)
print("test loss, test acc:", results)

Evaluate on test data
85/85 [==============================] - 22s 260ms/step - loss: 0.3541 - accuracy: 0.8264
test loss, test acc: [0.35411328077316284, 0.8263750076293945]


###**Rebuild sentences**

In [35]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)

def remove_padding_and_join(texts):
  for i in range(len(texts)):
    no_padding = filter(lambda a: a != '<PAD>', texts[i])
    texts[i] = ' '.join(list(no_padding))
  return texts
    

def reconstruct_predicted_list(texts):
  for i in range(len(texts)):
    pad_start = texts[i].index('<PAD>')
    texts[i] = texts[i][:pad_start]
    no_padding = filter(lambda a: a != '<PAD>', texts[i])
    texts[i] = ' '.join(list(no_padding))
  return texts

def get_predictions_on_test(count):
    Y_preds = model.predict([X_test[:count], X_test_decoder[:count]])
    encoded_argmax  = np.argmax(Y_preds, axis=-1)
    return encoded_argmax, Y_test[:count]

def get_predicted_sentences(predicted):
  predicted_word_list = list(map(sequence_to_text, predicted))
  predicted_sent_list = reconstruct_predicted_list(predicted_word_list)
  return predicted_sent_list

def get_original_sentences(original):
  original_word_list  = list(map(sequence_to_text, original))
  original_sent_list = remove_padding_and_join(original_word_list)
  return original_sent_list 


def rebuild_sentence_from_sequence(count=20):
  predicted, original = get_predictions_on_test(count)
  predicted_sent_list = get_predicted_sentences(predicted)
  original_sent_list  = get_original_sentences(original)
  return predicted_sent_list, original_sent_list
  


In [47]:
preds, origs = rebuild_sentence_from_sequence(10)
preds, origs

(['از زمان آغاز تازه\u200cترین دور جنگ میان مردان دو جناح فلسطینی که جریان مه گذشته تاکنون بیش از دویست نفر را برای برای را جو بودند کشته شده\u200cاند',
  'با این حال مقامات سازمان ملل می\u200cگویند سربازان مظنون وابسته به گردان نهم کنگو هنوز مورد بازجوئی قرارنگرفته\u200cاند',
  'با اینحال توفان در مناطق دیگر و مناطق ارتفاع موجب بروز سیل صدها و اینک میلیونها خانه به برق هستند',
  'در ارتش روز روز سه شنبه ارتش آمریکا گفت سربازان آمریکا رهبر سنی که که به او آمریکا را تصادفا کشتند',
  'قطار دوم هنوز که به گفته تصادف رسیده بود دست علاقه قرار گرفت و یا تأسیسات خارج شدن ولی معلوم معلوم نیست که آیا دو نیز شده بود یا نه',
  'نیکولا سارکوزی رییس جمهوری فرانسه از فرمان رییس جمهوری روسیه برای متوقف ساختن در در در گرجستان استقبال کرده است',
  'و این گزارش آمریکا که که گفته بد رفتاری\u200cها را گزارش داد نه رسانه\u200cهای خبری',
  'فرماندار ایالت ویرجینیا همچنین امروز در برای این این ایالت برای عزاداری برای قربانیان آزاد کرده است',
  'شهادت ژنرال پتروس و رایان کروکر به جلسه مشترک کمیته روابط پرزیدن

In [37]:
def log(args):
  for arg in args:
    print(arg, '\n')
    print('-----\n')

In [48]:
sos_id = last_idx
def infere_sentence(sentence):
  X_text = np.asarray([sentence], dtype=object)
  X_seq = tokenizer.texts_to_sequences(X_text)
  X = keras.preprocessing.sequence.pad_sequences(X_seq, maxlen=MAX_SEQ_LEN, padding='post')
  Y_pred = np.full((X.shape[0], 1), sos_id)
  for index in range(MAX_SEQ_LEN):
    X_decoder = keras.preprocessing.sequence.pad_sequences(Y_pred, maxlen=MAX_SEQ_LEN, padding='post')
    Y_pred_next = tf.argmax(model.predict([X, X_decoder]), axis=-1)[:, index:index+1]
    #log([X, Y_pred, X_decoder, Y_pred_next])
    Y_pred = np.concatenate([Y_pred, Y_pred_next], axis=1)
  return Y_pred

def predict_a_sentence(sentence):
  Y_pred = infere_sentence(sentence)
  output = np.delete(Y_pred, np.where(Y_pred==sos_id), axis=1)
  return get_predicted_sentences(output)

print(predict_a_sentence('مشابهی هم اکنون نیز لایحه مجلس در سنای آمریکا بحث وبررسی مورد قرار دارد'))
print(predict_a_sentence('بزرگ کشوری است ایران'))

['لایحه مشابهی نیز هم اکنون در مجلس سنای آمریکا مورد بحث وبررسی قرار دارد']
['ایران دارای فشار دارند']
